In [121]:
%matplotlib qt

from sklearnex import patch_sklearn
patch_sklearn()
import pandas as pd

df = pd.read_csv("../data/qbo_data.csv",index_col="run_id")
df = df.sort_values("qbo_period_std")


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [122]:
import numpy as np
X = df[["Bt","cw"]].to_numpy()
y_point_error = df[["qbo_period_std","qbo_amplitude_std"]].to_numpy()
y = df[["qbo_periods","qbo_amplitude_mean"]].to_numpy()

mean_y = np.mean(y,axis=0)
std_y = np.std(y,axis=0)
y_norm = (y - mean_y)/std_y
y_point_error = y_point_error/std_y

In [123]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

In [125]:
X_pca

array([[-1.66780167,  0.26229464],
       [ 0.18092344,  0.71834059],
       [ 0.05798028,  0.9866072 ],
       [ 1.79731204,  0.61989489],
       [ 0.61153785,  0.32673871],
       [ 0.09652892,  1.37800629],
       [ 0.56577836,  0.55146657],
       [ 0.71303057,  1.03119636],
       [-0.77289659,  0.36223925],
       [ 0.83522905, -0.17820458],
       [ 1.92265605,  0.33622453],
       [-0.14683059, -0.38915346],
       [-0.2539805 , -0.78602732],
       [ 1.22703528,  0.66897467],
       [ 0.0977235 , -0.58358592],
       [ 0.33513112, -0.11692471],
       [ 0.63223306, -0.52033936],
       [-0.32676226, -1.18000607],
       [ 0.89085266,  0.91485593],
       [-0.19805892,  0.22902775],
       [ 0.85719744,  1.32773939],
       [ 0.61449668, -0.63809845],
       [-1.54327993,  0.67960117],
       [ 0.8053951 , -0.47783868],
       [ 0.40244769,  0.74744954],
       [ 1.14324106,  0.98433498],
       [ 1.81203021,  0.00706241],
       [-0.45064579,  0.15254903],
       [-0.54026099,

In [126]:
from sklearn.gaussian_process import GaussianProcessRegressor

gpr_period = GaussianProcessRegressor(alpha=(y_point_error[:,0])**2)
gpr_amplitude = GaussianProcessRegressor(alpha=(y_point_error[:,1])**2)
gpr_period.fit(X_pca,y_norm[:,0])
gpr_amplitude.fit(X_pca,y_norm[:,1])


GaussianProcessRegressor(alpha=array([0.        , 0.03547776, 0.03204514, 0.02129178, 0.03661212,
       0.09994381, 0.04752571, 0.02339978, 0.06869182, 0.04361435,
       0.02058369, 0.0330511 , 0.029952  , 0.02990572, 0.03962432,
       0.04181666, 0.02632595, 0.03740381, 0.11036499, 0.05486589,
       0.11988828, 0.01287535, 0.07012233, 0.04252602, 0.01810179,
       0.02959773, 0.03893115, 0.03611464, 0.02273579, 0.05...
       0.03482906, 0.06321747, 0.124354  , 0.07802362, 0.42698006,
       0.05888453, 0.17278043, 0.05400357, 0.50627278, 0.11065289,
       0.19951574, 0.0018386 , 0.24543738, 0.11524   , 0.13536967,
       0.05538973, 0.58399116, 0.46842607, 0.11437035, 0.37871673,
       0.05733359, 0.45538299, 0.07037729, 0.04718549, 0.07573725,
       0.08398727, 0.0340759 , 0.04177656, 0.04106591, 0.60142414,
       0.14445963, 0.09313113, 0.0329861 , 0.06683739, 0.13460253]))

In [127]:
import numpy as np
cw = np.linspace(5,70,1000)
Bt = np.ones(len(cw))*0.0043
x_samples = np.array([Bt,cw]).T
x_samples = scaler.transform(x_samples)
x_samples = pca.transform(x_samples)


In [128]:
samples,samples_std = gpr_period.predict(x_samples,return_std=True)
samples = samples*std_y[0] + mean_y[0]

In [129]:
std_y

array([ 3.86400325, 10.59800016])

In [130]:
OBS_MEAN = 27.475168565819082
OBS_STD = 3.9921132486687365

OBS_AMP_MEAN = 26.68333333333334
OBS_AMP_STD = 2.7605353748060457

In [131]:
# import matplotlib.pyplot as plt
# fig,(ax1,ax2)= plt.subplots(nrows=1,ncols=2,figsize=(10,6))

# ax1.plot(cw,samples)
# ax1.fill_between(
#     cw,
#     samples - 1.96*samples_std*std_y[0],
#     samples + 1.96*samples_std*std_y[0],
#     color="tab:blue",
#     alpha=0.5,
#     label=r"95% confidence interval",
# )
# ax1.axhline(OBS_MEAN,color='black',label="True Observation")
# ax1.fill_between(cw,OBS_MEAN-OBS_STD,OBS_MEAN+OBS_STD,color='black',alpha=0.3)
# ax1.set_xlabel("Cw tropics")
# ax1.set_ylabel("QBO Period (months)")
# ax1.errorbar(X[:,1],y[:,0],yerr=y_point_error[:,0]*std_y[0],fmt='o',label="MiMA runs")
# ax1.set_ylim(10,40)
# ax1.legend()

# ax2.plot(cw,implausability)
# ax2.axhline(np.percentile(implausability,20),c='g',label=f"20th percentile implausability cut off ")
# ax2.legend()
# ax2.set_ylabel("1D Implausability")
# ax2.set_xlabel("cw")

### 3D Plots?

In [132]:
cw = np.linspace(5,70,100)
Bt = np.linspace(0.001,0.007,100)
X_Bt,Y_cw = np.meshgrid(Bt,cw)
x_samples = np.vstack((X_Bt.ravel(),Y_cw.ravel())).T
x_samples = scaler.transform(x_samples)
x_samples = pca.transform(x_samples)

In [133]:
x_samples

array([[-3.07522596e-03, -2.44899973e+00],
       [ 2.17004755e-02, -2.42422403e+00],
       [ 4.64761770e-02, -2.39944832e+00],
       ...,
       [-4.72288425e-02,  2.40163998e+00],
       [-2.24531410e-02,  2.42641568e+00],
       [ 2.32256049e-03,  2.45119138e+00]])

In [134]:
period,period_std = gpr_period.predict(x_samples,return_std=True)
period = period*std_y[0] + mean_y[0]
period = period.reshape((100,100))
period_std = (period_std*std_y[0]).reshape((100,100))


amplitude,amplitude_std = gpr_amplitude.predict(x_samples,return_std=True)
amplitude = amplitude*std_y[1] + mean_y[1]
amplitude = amplitude.reshape((100,100))
amplitude_std = (amplitude_std*std_y[1]).reshape((100,100))

In [135]:
implausability_period = np.abs(period - OBS_MEAN)/np.sqrt(OBS_STD**2 + period_std**2)
implausability_amplitude = np.abs(amplitude - OBS_AMP_MEAN)/np.sqrt(OBS_AMP_STD**2 + amplitude_std**2)
implausability = np.sqrt(implausability_amplitude**2 + implausability_period**2)

In [136]:
imp_cutoff = implausability < np.percentile(implausability,10)

In [142]:
fig,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(14,6))
CS = ax1.contour(X_Bt,Y_cw,amplitude,levels=25)
ax1.scatter(X[:,0],X[:,1],)
ax1.clabel(CS, CS.levels, inline=True,  fontsize=10)
ax1.set_ylabel("Cw")
ax1.set_xlabel("Bt")

CS2 = ax2.contour(X_Bt,Y_cw,period,levels=25)
ax2.scatter(X[:,0],X[:,1],)
ax2.clabel(CS2, CS2.levels, inline=True,  fontsize=10)
ax2.set_ylabel("Cw")
ax2.set_xlabel("Bt")

ax3.contour(X_Bt,Y_cw,implausability,levels=25)
ax3.contourf(X_Bt,Y_cw,imp_cutoff,alpha=0.3,cmap='RdYlGn')
ax3.scatter(X[:,0],X[:,1])
ax3.set_ylabel("Cw")
ax3.set_xlabel("Bt")

Text(0.5, 0, 'Bt')

In [138]:

from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
ax = plt.figure(figsize=(10,10)).add_subplot(projection='3d')
ax.plot_surface(X_Bt,Y_cw,samples,cmap='viridis')
ax.set_xlabel("Bt")
ax.set_ylabel("Cw")
ax.set_zlabel("QBO Period (months)")

ValueError: Argument Z must be 2-dimensional.

In [141]:
ax = plt.figure(figsize=(10,10)).add_subplot(projection='3d')
ax.plot_surface(X_Bt,Y_cw,implausability,cmap='viridis')
ax.set_xlabel("Bt")
ax.set_ylabel("Cw")
ax.set_zlabel("Implausability")

Text(0.5, 0, 'Implausability')